Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

### Инструкция по выполнению проекта

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

### Описание данных

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

# План работы:

1) [Изучение и обработка данных.](#id_1)

2) [Обучение моделей, подборка гиперпараметров.](#id_2)

3) [Вывод.](#id_3)


<a id='id_1'></a>
# 1. Подготовка

In [52]:
import pandas as pd
import re
import numpy as np
import nltk
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from lightgbm import LGBMClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import PassiveAggressiveClassifier 
from catboost import CatBoostClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv('/datasets/toxic_comments.csv')
data.info()
data.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
text     159571 non-null object
toxic    159571 non-null int64
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


У нас имеется: 159571 комментариев на английском языке, а так же колонка Toxic - которая является целевым признаком и отвечает на вопрос "Токсичен ли данный комментарий"
Текст на английском языке, он не обработан, его следует лемматизировать. Формат столбца - obj.

In [3]:
# Далее у меня зависал юпитер и ноутбук при работе с такой большой базой данных, хотя код был написан верно. 
# В результате чего мне пришлось сократить исходный датафрейм. Ошибка выдавалась при лемматизации.
data = data.sample(n=80000)

In [4]:
data = data.reset_index(drop=True)

In [5]:
# Напишем функции для очищения и лемматизации текста
corpus = list(data['text'])

def clear_text(text):
    text1 = re.sub(r'[^a-zA-Z ]', ' ', str(text))
    text1 = " ".join(text1.split()) 
    return text1

w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

def lemmatize_text(text):
    return ' '.join([lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)])


In [6]:
# Проверим, работают ли функции
lemmatize_text(clear_text(corpus[1]))

'Ukraine is Poland Ukraine isn t Russia or Poland Dude you are reported for highly offensive citation such a one above'

In [7]:
# Функции работают корректно, можем начинать обрабатывать датафрейм
data['lemma_text'] = 1
for i in range(len(data)):
    data.lemma_text[i] = lemmatize_text(clear_text(corpus[i]))
data.head(5)


,text,toxic,lemma_text
0,If these people are refugees fleeing persecuti...,0,If these people are refugee fleeing persecutio...
1,"""\n\n Ukraine is Poland. \n\n""""Ukraine isn't R...",0,Ukraine is Poland Ukraine isn t Russia or Pola...
2,"""\nI speak Spanish. Check out this!!. Wow, som...",0,I speak Spanish Check out this Wow someday may...
3,Frustrated Father \n\nEight years ago my wife ...,0,Frustrated Father Eight year ago my wife got a...
4,"""\n\n Please do not vandalize pages, as you di...",0,Please do not vandalize page a you did with th...


In [8]:
features = data['lemma_text']
target = data['toxic']
features_train, features_test, target_train, target_test = train_test_split(features, target, random_state = 12345, test_size=0.2)

In [9]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
tf_idf = count_tf_idf.fit_transform(features_train)
tf_idf2 = count_tf_idf.transform(features_test)

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Итак, данные загружены, обработаны и готовы к использованию. 

<a id='id_2'></a>
# 2. Обучение

## 2.1 Логистическая регрессия

In [10]:
model = LogisticRegression()
model.fit(tf_idf, target_train)
prediction = model.predict(tf_idf2)
f1_score(target_test, prediction)

0.7023764617125613

In [11]:
# создадим 2 списка, в которые мы будем добавлять результаты
models = []
results = []
models.append("LogisticRegression")
results.append('0.70')
results

['0.70']

## 2.2 Случайный Лес

In [12]:
for depth in range(1, 1002, 100):   
    model = RandomForestClassifier(max_depth=depth, random_state=123)
    model.fit(tf_idf, target_train)
    prediction = model.predict(tf_idf2)
    print(depth, f1_score(target_test, prediction))

1 0.0
101 0.3509803921568628
201 0.551405791019723
301 0.6248037676609105
401 0.681203007518797
501 0.6740988480118916
601 0.6885608856088561
701 0.6830340784170026
801 0.6851032448377581
901 0.6616199848599545
1001 0.6741488963711185


In [13]:
for est in range (1, 402, 100):
    model = RandomForestClassifier(max_depth=601, n_estimators=est, random_state=123)
    model.fit(tf_idf, target_train)
    prediction = model.predict(tf_idf2)
    print(est, f1_score(target_test, prediction))

1 0.5553447185325744
101 0.703288490284006
201 0.713276310896244
301 0.7156644394951743
401 0.7157738095238095


In [15]:
models.append('RandomForest')
results.append('0.71')

## 2.3 LightGBM

In [16]:
# Подбираем параметры
for est in range (21, 72, 10):
    model=LGBMClassifier(n_estimators=est)
    model.fit(tf_idf, target_train)
    prediction = model.predict(tf_idf2)
    print(est, f1_score(target_test, prediction))

21 0.6535036778939218
31 0.6829451540195342
41 0.6990791896869245
51 0.7147016011644832
61 0.7230046948356806
71 0.7307829817661781


In [17]:
for depth in range (1, 202, 100):
    model=LGBMClassifier(n_estimators=71, max_depth=depth)
    model.fit(tf_idf, target_train)
    prediction = model.predict(tf_idf2)
    print(depth, f1_score(target_test, prediction))

1 0.3799421407907425
101 0.7307829817661781
201 0.7307829817661781


KeyboardInterrupt: 

In [18]:
models.append('LightGBM')
results.append('0.73')

## 2.4 DecissionTree

In [34]:
for depth in range(1, 302, 50):
    model = DecisionTreeClassifier(max_depth=depth, random_state=123)
    model.fit(tf_idf, target_train)
    prediction = model.predict(tf_idf2)
    print(depth, f1_score(target_test, prediction))


1 0.278974358974359
51 0.7147766323024054
101 0.7270349787512259
151 0.7307692307692308
201 0.7229601518026566
251 0.723125
301 0.7206427688504327


In [35]:
for split in range(300, 601, 50):
    model = DecisionTreeClassifier(max_depth=151, random_state=123, min_samples_split=split)
    model.fit(tf_idf, target_train)
    prediction = model.predict(tf_idf2)
    print(split, f1_score(target_test, prediction))

300 0.7212806026365348
350 0.7250547388176415
400 0.7240839336047604
450 0.7240839336047604
500 0.7244834063869755
550 0.7244834063869755
600 0.7244834063869755


In [57]:
models.append('DecissionTree')
results.append('0.73')

## 2.5 CatBoost

In [30]:
model=CatBoostClassifier(iterations=400)
model.fit(tf_idf, target_train)
prediction = model.predict(tf_idf2)
f1_score(target_test, prediction)

Learning rate set to 0.122981
0:	learn: 0.5767293	total: 3.38s	remaining: 22m 28s
1:	learn: 0.4894193	total: 6.17s	remaining: 20m 27s
2:	learn: 0.4250355	total: 8.77s	remaining: 19m 21s
3:	learn: 0.3762148	total: 11.5s	remaining: 18m 56s
4:	learn: 0.3395581	total: 14.3s	remaining: 18m 46s
5:	learn: 0.3133912	total: 17s	remaining: 18m 34s
6:	learn: 0.2938120	total: 19.7s	remaining: 18m 23s
7:	learn: 0.2789433	total: 22.4s	remaining: 18m 15s
8:	learn: 0.2671839	total: 25.1s	remaining: 18m 9s
9:	learn: 0.2562324	total: 28s	remaining: 18m 10s
10:	learn: 0.2486485	total: 30.7s	remaining: 18m 4s
11:	learn: 0.2429889	total: 33.4s	remaining: 17m 58s
12:	learn: 0.2381412	total: 36.1s	remaining: 17m 53s
13:	learn: 0.2340736	total: 38.8s	remaining: 17m 48s
14:	learn: 0.2303018	total: 41.5s	remaining: 17m 44s
15:	learn: 0.2267575	total: 44.3s	remaining: 17m 42s
16:	learn: 0.2238710	total: 47.1s	remaining: 17m 40s
17:	learn: 0.2207631	total: 49.9s	remaining: 17m 38s
18:	learn: 0.2186865	total: 52.6

0.7295552367288379

In [58]:
models.append('CatBoost')
results.append('0.73')

## 2.6 PassiveAggressiveClassifier 

In [51]:
model = PassiveAggressiveClassifier(random_state=12345, max_iter=5)
model.fit(tf_idf, target_train)
prediction = model.predict(tf_idf2)
f1_score(target_test, prediction)

0.761319534282018

In [59]:
models.append('PassiveAggressive')
results.append('0.76')

In [66]:
columns = ['F1']
all_results = pd.DataFrame(results, index=models, columns=columns)

<a id='id_3'></a>
# 3. Выводы

In [67]:
all_results

,F1
LogisticRegression,0.70
RandomForest,0.71
LightGBM,0.73
DecissionTree,0.73
CatBoost,0.73
PassiveAggressive,0.76


## Вывод:

Условием данного задания было составить модель с метрикой F1 не менее 0.75. Из всех подобранных моделей подходит только PassiveAggressive. Модели CatBoost, DecissionTree и LightGBM идут примерно одинаково с показателем 0.73, но для уровня задачи этого недостаточно. Еще чуть хуже себя показали модели RandomForest и LogisticRegression (0.71 и 0.70). 